#### Silhouette

##### Wygenerowanie danych

In [ ]:
import pandas as pd
from sklearn.datasets import make_blobs

data = make_blobs(n_samples=1000, centers=3, cluster_std=1.0, center_box=(-4.0, 4.0), random_state=42)[0]
df = pd.DataFrame(data, columns=['x1', 'x2'])
df.head()

In [ ]:
import plotly.express as px

px.scatter(df, 'x1', 'x2', width=950, height=500, title='Algorytm K-średnich', template='plotly_dark')

Przyjmujemy pięć klastrów

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5)
kmeans.fit(data)

In [ ]:
# przypisania
y_kmeans = kmeans.predict(data)
df['y_kmeans'] = y_kmeans

In [ ]:
# wizualizacja
px.scatter(
    df, 
    'x1', 
    'x2', 
    'y_kmeans', 
    width=950, 
    height=500, 
    title='Algorytm K-średnich - 5 klastrów', 
    template='plotly_dark'
)

Policzmy ręcznie silhouette

In [ ]:
import numpy as np
from numpy.linalg import norm

def dist_to_cluster(point: np.array, cluster: pd.DataFrame) -> float:
    """ Calculates the distance between the given point and the given cluster as a sum of distances between the given point 
    and every point in the given cluster."""
    
    distance = 0
    for index, row in cluster.iterrows():
        cluster_point = np.array([row.x1, row.x2])
        distance += norm(point - cluster_point)
    return distance

def a(point: np.array, cluster: pd.DataFrame) -> float:
    """Calculates the average distance between the given point and all other points within the same cluster as the given point."""
    
    distances = dist_to_cluster(point, cluster)
    cluster_length = cluster.shape[0]
    return 1 / (cluster_length - 1) * distances

def b(point: np.array, other_clusters: list[pd.DataFrame]) -> float:
    """Calculates the average distance between the given point and all points from the nearest cluster to the cluster of the given point."""
    
    result = [1 / cluster.shape[0] * dist_to_cluster(point, cluster) 
              for cluster 
              in other_clusters]
    return min(result)  # result for the nearest cluster

def silhouette(point: np.array, cluster: pd.DataFrame, other_clusters: list[pd.DataFrame]) -> float:
    """Calculates silhouette metric for the given point."""
    
    cluster_length = cluster.shape[0]
    if cluster_length == 1:
        return 0
    else:
        ar = a(point, cluster)
        br = b(point, other_clusters)
        return (br - ar) / max(ar, br)

def silhouette_score(clusters: dict[int, pd.DataFrame]) -> float:
    """Calculates the silhouette score for the given list of clusters."""
    
    if len(clusters) == 1:
        return 0
        
    results = []
    for cluster_nr, cluster in clusters.items():
        print(f"Cluster number: {cluster_nr}")
        other_clusters = [c for c_nr, c in clusters.items() if c_nr != cluster_nr]
        for index, row in cluster.iterrows():
            point = np.array([row.x1, row.x2])
            s = silhouette(point, cluster, other_clusters)
            results.append(s)
    return np.mean(results)

In [ ]:
cluster_1 = df[df.y_kmeans==0]
cluster_2 = df[df.y_kmeans==1]
cluster_3 = df[df.y_kmeans==2]
cluster_4 = df[df.y_kmeans==3]
cluster_5 = df[df.y_kmeans==4]

clusters = {
    1: cluster_1, 
    2: cluster_2, 
    3: cluster_3, 
    4: cluster_4, 
    5: cluster_5
}  # because comparing clusters wouldn't be efficience
result = silhouette_score(clusters)
print(result)

I znów, można ale nie trzeba liczyć tego ręcznie, bo w module metrics biblioteki sklearn mamy zaimplementowaną miarę silhouette za pomocą funkcji silhouette_score.

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_avg = silhouette_score(data, y_kmeans)
print(silhouette_avg)  # drobne niedokładności z uwagi na arytmetykę zmiennoprzecinkową

Metryka silhouette przyjmuje wartości z zakresu [-1, 1]. Im wyższa wartość silhouette tym lepsza jakość klasteryzacji.

#### Davies-Bouldin Index

Algorytmu do obliczania indeksu DB nie będziemy już samodzielnie implementować, ale zadanie jest porównywalne z poprzednim i może być dobrym ćwiczeniem do wykonania. Moduł metrics posiada dużą liczbę metryk ewaluacji modelu.

In [ ]:
import sklearn.metrics

print(dir(sklearn.metrics))

Wśród nich znajduję się również Davies-Bouldin Index. Oblicza go funkcja davies_bouldin_score

In [ ]:
from sklearn.metrics import davies_bouldin_score

result = davies_bouldin_score(data, y_kmeans)
print(result)

Im mniejsza wartość DBI tym lepsza jakość klasteryzacji.

Zróbmy analizę liczby klastrów dla obu metryk.